# PyMAST Complete Project Workflow
## Movement Analysis Software for Telemetry Data

![PyMAST Logo](pymast_logo.png)

---

## 🎯 What This Notebook Does

This notebook will guide you through analyzing your radio telemetry data from start to finish. **No programming experience required!** Just follow along, update the values in CAPITAL LETTERS with your own information, and press the "Run" button (▶️) on each cell.

### What You'll Accomplish:

1. **Load your project** - Connect to your fish tracking data
2. **Import receiver files** - Bring in detection data from the field
3. **Clean the data** (optional) - Remove false detections using AI
4. **Find fish visits** - Identify when fish were at each location
5. **Resolve overlaps** - Handle fish detected by multiple receivers
6. **Export for analysis** - Create files for statistical software

### ⏱️ Time Required: 
- **First time**: 2-4 hours (depending on data size)
- **Repeat sessions**: 5-10 minutes (just run setup cells)

### 💾 What You Need Before Starting:

Create a folder on your computer with these files:
- `tblMasterTag.csv` - Information about your fish tags (tag IDs, pulse rates, etc.)
- `tblMasterReceiver.csv` - Information about your receivers (receiver IDs, locations)
- `tblNodes.csv` - Locations/areas you're studying (optional)
- A folder called `Data` with a subfolder `Raw_Files` containing your receiver download files

### 📖 How to Use This Notebook:

1. **Read the text in each section** - It explains what's happening
2. **Look for 🔧 EDIT THIS** markers - These tell you what to change
3. **Click the cell and press Shift+Enter** - This runs the code
4. **Wait for the ✓ checkmark** - This means it worked!
5. **If you see an error** - Read it carefully, it usually tells you what's wrong

### 💡 Pro Tips:

- **Run cells in order** - Don't skip around or things will break
- **Save often** - Use Ctrl+S or File → Save
- **You can't break anything permanently** - The database keeps backups
- **Green text starting with #** - These are comments, just notes for you
- **Lines starting with #** - Won't run, they're turned off (you can turn them on by removing the #)

---

## 🚀 Ready? Let's Start!

**First step**: Scroll down to Section 1.1 and click the Run button (▶️) to load the software.

# Part 1: Project Setup

**Run these cells every session** to load modules and connect to your project database.

## 1.1 Import Required Modules

In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Ensure PyMAST is importable
try:
    import pymast
except ImportError:
    # Add PyMAST to path if not installed via pip
    repo_root = Path.cwd().parent if 'scripts' in str(Path.cwd()) else Path.cwd()
    sys.path.insert(0, str(repo_root))
    import pymast

from pymast.radio_project import radio_project
from pymast import formatter
from pymast.overlap_removal import bout, overlap_reduction

print(f"PyMAST version: {pymast.__version__ if hasattr(pymast, '__version__') else 'unknown'}")
print(f"Python executable: {sys.executable}")

## 1.2 Initialize or Load Project

**First time:** Creates new HDF5 database with your metadata  
**Subsequent sessions:** Reconnects to existing database

**Required files in `project_dir`:**
- `tblMasterTag.csv` - Tag metadata (freq_code, pulse_rate, tag_type, etc.)
- `tblMasterReceiver.csv` - Receiver metadata (rec_id, rec_type, node, lat/lon)
- `tblNodes.csv` - Spatial nodes for movement modeling (optional)

In [ ]:
# PROJECT CONFIGURATION
project_dir = r"C:\path\to\your\project"  # Update this path!
db_name = 'my_telemetry_study'

# Load metadata
tag_data = pd.read_csv(os.path.join(project_dir, 'tblMasterTag.csv'))
receiver_data = pd.read_csv(os.path.join(project_dir, 'tblMasterReceiver.csv'))
nodes_data = pd.read_csv(os.path.join(project_dir, 'tblNodes.csv'))

# Initialize project
project = radio_project(
    project_dir=project_dir,
    db_name=db_name,
    detection_count=5,  # Minimum detections for valid bout
    duration=1,         # Time window for detection clustering (minutes)
    tag_data=tag_data,
    receiver_data=receiver_data,
    nodes_data=nodes_data
)

print(f"\n✓ Project initialized: {project.db}")
print(f"  Tags: {len(tag_data)}")
print(f"  Receivers: {len(receiver_data)}")
print(f"  Nodes: {len(nodes_data)}")

### Optional: Create New Database Version

Uncomment to create a new version while preserving original data.

In [ ]:
# project.new_db_version(os.path.join(project_dir, f'{db_name}_v2.h5'))

---

# Part 2: Data Import

**Run once per receiver** to import raw detection files.

## 2.1 Import Receiver Data

**Supported receiver types:**
- Lotek: `srx1200`, `srx800`, `srx600`
- Sigma Eight: `orion`, `ares`
- Vemco: `vr2`

**Update these parameters for each receiver:**

In [ ]:
# RECEIVER IMPORT CONFIGURATION
rec_id = 'REC001'              # Receiver ID from tblMasterReceiver.csv
rec_type = 'srx1200'           # Receiver type
scan_time = 2.5                # Scan duration per channel (seconds)
channels = 1                   # Number of receiver channels
antenna_to_rec_dict = {'A0': rec_id}  # Antenna to receiver mapping

# Directory containing raw detection files
data_dir = os.path.join(project_dir, 'Data', 'Raw_Files')
db_dir = os.path.join(project_dir, f'{db_name}.h5')

# Import data
project.telem_data_import(
    rec_id=rec_id,
    rec_type=rec_type,
    file_dir=data_dir,
    db_dir=db_dir,
    scan_time=scan_time,
    channels=channels,
    ant_to_rec_dict=antenna_to_rec_dict
)

print(f"\n✓ Imported data for {rec_id}")

### Undo Import (if needed)

Run only if you need to re-import with different parameters:

In [ ]:
# project.undo_import(rec_id)

---

# Part 3: Classification (Optional)

Train a Naive Bayes classifier to identify and remove false positive detections.

**Skip this section if:**
- You don't have hand-labeled training data
- False positives are minimal in your dataset

## 3.1 Train Classifier

**Run once per receiver** after creating training data via manual labeling.

In [ ]:
# TRAINING CONFIGURATION
rec_id = 'REC001'
rec_type = 'srx1200'

# Get list of fish detected at this receiver
fishes = project.get_fish(rec_id=rec_id)
print(f"Training classifier for {len(fishes)} fish at {rec_id}...")

# Train classifier for each fish
for fish in fishes:
    project.train(fish, rec_id)

# Generate training summary statistics
project.training_summary(rec_type, site=[rec_id])

print("\n✓ Training complete")

### Undo Training (if needed)

In [ ]:
# project.undo_training(rec_id)

## 3.2 Classify Detections

Apply trained classifier to identify false positives.

**Likelihood predictors** (À la carte - choose any combination):
- `hit_ratio`: Proportion of detections matching expected pulse intervals
- `cons_length`: Maximum contiguous sequence of expected detections
- `noise_ratio`: Ratio of miscoded to total detections
- `power`: Signal strength
- `lag_diff`: Variability in inter-detection intervals

In [ ]:
# CLASSIFICATION CONFIGURATION
rec_id = 'REC001'
rec_type = 'srx1200'
threshold_ratio = 1.0  # 1.0 = MAP hypothesis (balanced), >1.0 = stricter, <1.0 = more permissive

# Select likelihood predictors
likelihood = ['hit_ratio', 'cons_length', 'noise_ratio', 'power', 'lag_diff']

# Classify detections
project.reclassify(
    project=project,
    rec_id=rec_id,
    rec_type=rec_type,
    threshold_ratio=threshold_ratio,
    likelihood_model=likelihood
)

print("\n✓ Classification complete")

### Undo Classification (if needed)

In [ ]:
# project.undo_classification(rec_id)

---

# Part 4: Bout Detection with DBSCAN

Identify continuous presence periods using density-based clustering.

**DBSCAN Parameters:**
- `eps_multiplier`: Sets temporal threshold (epsilon = pulse_rate × eps_multiplier)
  - Default: 5 (~40-50 seconds for typical 8-10 sec tags)
  - Higher values = longer gaps allowed within bouts
- `lag_window`: Legacy parameter (kept for compatibility, not used in clustering)

## 4.1 Run Bout Detection for All Receivers

In [ ]:
# Optionally undo existing bouts if re-running
# project.undo_bouts()

print("=" * 80)
print("BOUT DETECTION (DBSCAN)")
print("=" * 80)

receivers = list(project.receivers.index)
print(f"\nProcessing {len(receivers)} receivers...\n")

successful_receivers = []
failed_receivers = []

for rec_id in receivers:
    try:
        print(f"[{rec_id}] Starting bout detection...")
        
        # Create bout object - DBSCAN runs automatically during init
        bout_obj = bout(
            radio_project=project,
            rec_id=rec_id,
            eps_multiplier=5,  # 5× pulse rate for temporal clustering
            lag_window=9       # Legacy parameter (not used in DBSCAN)
        )
        
        # Cluster detections into bouts
        bout_obj.cluster()
        
        # Visualize bout lengths (creates diagnostic plots)
        bout_obj.visualize_bout_lengths()
        
        successful_receivers.append(rec_id)
        print(f"[{rec_id}] ✓ Complete\n")
        
    except Exception as e:
        failed_receivers.append(rec_id)
        print(f"[{rec_id}] ✗ Failed: {e}\n")

print("\n" + "=" * 80)
print(f"Bout Detection Summary:")
print(f"  Successful: {len(successful_receivers)}")
print(f"  Failed: {len(failed_receivers)}")
if failed_receivers:
    print(f"  Failed receivers: {', '.join(failed_receivers)}")
print("=" * 80)

### Undo Bouts (if needed)

Undo all bouts or specific receiver:

In [ ]:
# Undo all bouts
# project.undo_bouts()

# Undo specific receiver
# project.undo_bouts(rec_id='REC001')

---

# Part 5: Overlap Resolution

Resolve spatially ambiguous detections when fish are detected by multiple receivers simultaneously.

**Two methods available:**
1. **Unsupervised** - Statistical comparison (t-test + Cohen's d) of signal power/posterior
2. **Nested Doll** - Hierarchical removal based on parent-child receiver relationships

## 5.1 Unsupervised Overlap Resolution (Recommended)

Uses statistical testing to identify stronger signal receiver.

**Parameters:**
- `method`: `'power'` or `'posterior'` for comparison metric
- `p_value_threshold`: Require statistical significance (default: 0.05)
- `effect_size_threshold`: Minimum Cohen's d (default: 0.2 = small effect)
- `min_detections`: Minimum bout size for comparison (default: 5)
- `bout_expansion`: Time buffer for near-overlaps in seconds (default: 60)

In [ ]:
import time

# Define network structure
nodes = list(project.receivers.index)
edges = [(i, j) for i in nodes for j in nodes if i != j]  # All possible receiver pairs

print("Initializing overlap reduction...")
start_time = time.time()

# Create overlap reduction object
overlap_obj = overlap_reduction(
    nodes=nodes,
    edges=edges,
    radio_project=project
)

# Run unsupervised removal with statistical testing
print("Running unsupervised overlap resolution...")
overlap_obj.unsupervised(
    # method='power',              # Compare signal power
    # p_value_threshold=0.05,      # Require p < 0.05
    # effect_size_threshold=0.2,   # Small effect size
    # min_detections=5,            # Minimum bout size
    # bout_expansion=60            # ±60 second buffer
)

# Visualize overlap analysis (creates 8-panel diagnostic plot)
overlap_obj.visualize_overlaps()

elapsed = time.time() - start_time
print(f"\n✓ Overlap resolution complete ({elapsed:.1f} seconds)")

## 5.2 Nested Doll Overlap Resolution (Alternative)

Hierarchical removal based on known parent-child receiver relationships.

**Use when:**
- You have directional antennas with known coverage hierarchy
- Small dipole receivers within larger Yagi coverage areas
- Clear spatial containment relationships

In [ ]:
# # Define parent:child relationships (outer receiver : inner receiver)
# edges = [
#     ('R04', 'R15'), ('R04', 'R14'), ('R04', 'R13'),  # R04 contains R15, R14, R13...
#     ('R03', 'R15'), ('R03', 'R14'), ('R03', 'R13'),  # R03 contains R15, R14, R13...
#     ('R08', 'R10'),                                   # R08 contains R10
#     ('R09', 'R10')                                    # R09 contains R10
# ]
# nodes = ['R03', 'R04', 'R05', 'R08', 'R09', 'R10', 'R12', 'R13', 'R14', 'R15']

# # Create overlap object and apply nested doll algorithm
# nested_obj = overlap_reduction(
#     nodes=nodes,
#     edges=edges,
#     radio_project=project
# )
# nested_obj.nested_doll()

# print("\n✓ Nested doll overlap resolution complete")

### Undo Overlap Resolution (if needed)

In [ ]:
# project.undo_overlap()

---

# Part 6: Create Recaptures Table

Aggregate all detections into spatially/temporally resolved recaptures table.

**This table:**
- Links detections to spatial nodes
- Applies classification and overlap decisions
- Serves as input for all statistical models

In [ ]:
# Create recaptures table
project.make_recaptures_table(export=True)

print("\n✓ Recaptures table created")
print(f"  Exported to: {os.path.join(project_dir, 'Output', 'recaptures.csv')}")

### Undo Recaptures (if needed)

Optionally repack database after undo to reclaim disk space:

In [ ]:
# project.undo_recaptures()
# project.repack_database()  # Reclaim disk space

---

# Part 7: Movement Analysis

Export data for survival/movement models.

**Available formats:**
1. **Time-to-Event (TTE)** - Competing risks, multi-state models (R survival package)
2. **Cormack-Jolly-Seber (CJS)** - Mark-recapture (Program MARK/RMark)
3. **Live-Recapture Dead-Recovery (LRDR)** - Dual recovery models

## 7.1 Time-to-Event Model

Export data for competing risks and multi-state Markov models.

**Step 1:** Define node-to-state mapping

In [ ]:
# Map spatial nodes to model states
node_to_state = {
    'R15': 1, 'R14': 1,              # Occum Pond (upstream)
    'R13': 2, 'R12': 2,              # Downstream gate
    'R10': 3,                         # Tailrace
    'R11': 4,                         # Fish lift entrance
    'R08': 5, 'R09': 5,              # Spillway
    'R06': 6, 'R07': 6,              # Surface bypass
    'R05': 7,                         # Submerged bypass
    'R04': 8,                         # Fish lift exit
    'R03': 9,                         # Forebay
    'R01': 10, 'R02': 10             # Windham (downstream)
}

print(f"Defined {len(set(node_to_state.values()))} states from {len(node_to_state)} receivers")

**Step 2:** Create time-to-event data object

**Optional parameters:**
- `initial_state_release=True`: Add state 0 for release location (models fallback)
- `last_presence_time0=True`: Use last detection at initial receiver as t=0 (for downstream migration)
- `cap_loc`: Filter by capture location
- `rel_loc`: Filter by release location
- `species`: Filter by species (if multi-species study)

In [ ]:
# Create TTE data object
tte = formatter.time_to_event(
    node_to_state=node_to_state,
    radio_project=project,
    initial_state_release=True,   # Add release state (state 0)
    last_presence_time0=False,    # t=0 at first detection
    cap_loc=None,                 # All capture locations
    rel_loc='tailrace',           # Filter by release location
    species=None,                 # All species
    recap_date=None               # All dates
)

print("✓ TTE data object created")

**Step 3:** Prepare data with adjacency filter

**Adjacency filter** removes impossible movements (e.g., forebay → tailrace without passing through dam).

Specify illegal (from_state, to_state) transitions:

In [ ]:
# Define illegal state transitions (from_state, to_state)
adjacency_filter = [
    (9, 1), (9, 2), (9, 3),  # Forebay can't jump to upstream states
    (8, 1), (8, 2), (8, 3),  # Lift exit can't jump to upstream
    (1, 8), (1, 9),          # Upstream can't teleport to forebay/exit
    (2, 9), (2, 8),          # Gate can't skip to forebay/exit
    (3, 8), (3, 9)           # Tailrace can't jump to forebay/exit
]

# Prepare data (applies adjacency filter)
tte.data_prep(
    radio_project=project,
    adjacency_filter=adjacency_filter
)

# Generate summary statistics
stats = tte.summary()

print("\n✓ TTE data preparation complete")
print(f"  Movement summary: {os.path.join(project_dir, 'Output', 'movement_summary.csv')}")
print(f"  State table: {os.path.join(project_dir, 'Output', 'state_table.csv')}")

### View TTE Output Tables

In [ ]:
# Display TTE output tables
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

output_dir = os.path.join(project_dir, "Output")
df_movement = pd.read_csv(os.path.join(output_dir, "movement_summary.csv"))
df_state = pd.read_csv(os.path.join(output_dir, "state_table.csv"))

print("=== Movement Summary ===")
print(df_movement)
print("\n=== State Table (first 10 rows) ===")
print(df_state.head(10))

## 7.2 Cormack-Jolly-Seber Model

Simple mark-recapture analysis (e.g., fish ladder effectiveness).

**Step 1:** Map receivers to recapture occasions

In [ ]:
# Map receivers to recapture occasions
receiver_to_recap = {
    'R01': 'R01', 'R02': 'R01',              # Occasion 1: Downstream
    'R10': 'R02', 'R12': 'R02',              # Occasion 2: Tailrace
    'R13': 'R03', 'R14': 'R03',              # Occasion 3: Dam
    'R15': 'R04', 'R16': 'R04',              # Occasion 4: Upstream
    'R03': 'R05', 'R04': 'R05'               # Occasion 5: Forebay
}

model_name = "my_study_cjs"
output_ws = os.path.join(project_dir, 'Output')

print(f"Defined {len(set(receiver_to_recap.values()))} recapture occasions")

**Step 2:** Create CJS input file for Program MARK

In [ ]:
# Create CJS data object
cjs = formatter.cjs_data_prep(
    receiver_to_recap=receiver_to_recap,
    radio_project=project,
    species=None,                    # All species
    initial_recap_release=False      # First detection is first recapture
)

# Generate MARK input file
cjs.input_file(model_name, output_ws)
cjs.inp.to_csv(os.path.join(output_ws, f'{model_name}.csv'), index=False)

print("\n✓ CJS input file created")
print(f"  INP file: {os.path.join(output_ws, f'{model_name}.inp')}")
print(f"  CSV file: {os.path.join(output_ws, f'{model_name}.csv')}")
print("\n  Next step: Import .inp file into Program MARK or RMark")

---

# Part 8: Quality Control & Visualization

Visual inspection of fish movement histories.

## 8.1 Fish Movement Histories (3D Plots)

In [ ]:
# from pymast.fish_history import fish_history

# # Load fish tracks
# fh = fish_history(
#     projectDB=project.db,
#     filtered=True,          # Show only filtered detections (test==0)
#     overlapping=False       # Exclude overlapping detections
# )

# # View specific fish
# fh.change_fish('166.380 7')
# fh.plot_3d_trajectory()

# print("\n✓ Fish history visualization complete")

---

# Summary

**Your PyMAST project is complete!**

**Output files created:**
- `{db_name}.h5` - HDF5 database with all processed data
- `Output/movement_summary.csv` - TTE movement summary
- `Output/state_table.csv` - TTE state transitions
- `Output/recaptures.csv` - Spatiotemporal recaptures table
- `Output/{model_name}.inp` - MARK input file (if CJS created)

**Next steps:**
1. Import TTE data into R (`survival` package) for competing risks models
2. Import CJS .inp file into Program MARK or RMark for mark-recapture analysis
3. Review diagnostic plots for quality control
4. Iterate on adjacency filters if impossible movements detected

**Documentation:**
- [GETTING_STARTED.md](GETTING_STARTED.md) - Complete setup guide
- [ARCHITECTURE.md](ARCHITECTURE.md) - System overview and database structure
- Python help system: `help(pymast.module_name)`

---

**PyMAST** - Movement Analysis Software for Telemetry  
https://github.com/knebiolo/mast